# Function Calling Tutorial

Function calling lets an LLM:
- Detect when a user request should call your function/tool.
- Return structured arguments (not just text) that your code can directly run.
- Receive the results back and continue the conversation.

User → LLM (decides what function + args) → Your code runs → Result → LLM formats answer.

In [22]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [32]:
# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get today's weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "A city name like Sydney or New York",
                },
            },
            "required": ["city"],
        },
    },
]

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is the weather in Sydney today?"}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-3.5-turbo",
    tools=tools,
    input=input_list,
)

print(response.model_dump_json(indent=2))

{
  "id": "resp_68b05df93958819f999ce35e78b30c86039fe82a55df32a1",
  "created_at": 1756388857.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "arguments": "{\"city\":\"Sydney\"}",
      "call_id": "call_WUgQf9pmDQpFqI6V9O2An24m",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68b05dfb5b1c819fa04dc5e786d87b08039fe82a55df32a1",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "A city name like Sydney or New York"
          }
        },
        "required": [
          "city"
        ],
        "additionalProperties": false
      },
      "strict": true,
      "type": "functio

In [24]:
# Save function call outputs for subsequent requests
function_call = None
function_call_arguments = None
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        print(function_call)
        function_call_arguments = json.loads(item.arguments)
        print(function_call_arguments)

ResponseFunctionToolCall(arguments='{"city":"Sydney"}', call_id='call_zifTCdhM5pxWzhhp6KijzeL9', name='get_weather', type='function_call', id='fc_68b05d083020819293fa14390b4a1d620bc922bc27648d32', status='completed')
{'city': 'Sydney'}


In [25]:
# Dummy function implementation
def get_weather(city):
    if city.lower() == "sydney":
        return "24°C, partly cloudy with a light breeze."
    else:
        return "2°C, thunderstorms."

# 3. Execute the function logic for get_weather
result = {"weather": get_weather(function_call_arguments["city"])}
print(result)

{'weather': '24°C, partly cloudy with a light breeze.'}


In [26]:
# 4. Provide function call results to the model
input_list.append({
    "type": "function_call_output",
    "call_id": function_call.call_id,
    "output": json.dumps(result),
})

print("Final input:")
print(input_list)

Final input:
[{'role': 'user', 'content': 'What is the weather in Sydney today?'}, ResponseFunctionToolCall(arguments='{"city":"Sydney"}', call_id='call_zifTCdhM5pxWzhhp6KijzeL9', name='get_weather', type='function_call', id='fc_68b05d083020819293fa14390b4a1d620bc922bc27648d32', status='completed'), {'type': 'function_call_output', 'call_id': 'call_zifTCdhM5pxWzhhp6KijzeL9', 'output': '{"weather": "24\\u00b0C, partly cloudy with a light breeze."}'}]


In [ ]:
response = client.responses.create(
    model="gpt-3.5-turbo",
    instructions="Respond only with the weather generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print(response.model_dump_json(indent=2))
print("\n\n" + "Final output:")
print(response.output_text)


Final output:
{
  "id": "resp_68b05d08bd7881929f30834fe546179d0bc922bc27648d32",
  "created_at": 1756388616.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with the weather generated by a tool.",
  "metadata": {},
  "model": "gpt-3.5-turbo-0125",
  "object": "response",
  "output": [
    {
      "id": "msg_68b05d0996d88192ba7bc8c9e8ad039a0bc922bc27648d32",
      "content": [
        {
          "annotations": [],
          "text": "The weather in Sydney today is 24°C, partly cloudy with a light breeze.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "A city name 

2. Single-round tool execution (automatic function calling)

In [42]:
response = client.responses.create(
    model="gpt-5",
    input=[{"role": "user", "content": "What is the weather in Sydney?"}],
    tools=tools,
    tool_choice="auto"   # let it auto-call
)
print(response.model_dump_json(indent=2))
print("\n\n" + "Final output:")
print(response.output_text)

{
  "id": "resp_68b05ef10584819d96ed20a56310f90909476275909c468d",
  "created_at": 1756389105.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "rs_68b05ef1631c819da9488d5504ea614409476275909c468d",
      "summary": [],
      "type": "reasoning",
      "content": null,
      "encrypted_content": null,
      "status": null
    },
    {
      "arguments": "{\"city\":\"Sydney\"}",
      "call_id": "call_tVQwioqMIxGDCkCDiew7uGEZ",
      "name": "get_weather",
      "type": "function_call",
      "id": "fc_68b05ef2273c819dad144f4d731813e509476275909c468d",
      "status": "completed"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_weather",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description":